# FIT5196 Assessment 1 - Task 1

Date: 14/04/2019

Version: 1.0

Environment: Python 3.6.5 and Anaconda 4.4.10 (64-bit)

Libraries used:
* re 2.2.1 (for regular expression, included in Anaconda Python 3.6) 
* json 2.0.9 (for dumping the data in json format, included in Anaconda Python 3.6)

## 0. Assumption
The input file `29620163.txt` is in the current directory. 


## 1. Introduction
This assignament needs the html data given to be parsed using regular expressions and dump the data into required json and xml format. The input data is in the file `29620163.txt` The required tasks are the following:

1. Read the input file and split the entire data into list of units using Regular expression.
2. For each unit chunk, use various regular expressions to extract the relavant information and store in a dictionary.
3. Dump the dictionary formed in the json output file using the json package.
4. Iterate over the dictionary to write the required output file. 


More details for each task will be given in the following sections.

## 2.  Import libraries 

In [1]:
import re
import json

## 3. Examining and loading data

We read the file, and the first few characters are displayed.

In [2]:
with open('29620163.txt', 'r') as testfile:
    my_str = testfile.read()
my_str[0:1000]

'<div class="content-inner__main">\n<!-- breadcrumbs -->\n<nav class="breadcrumbs mobile-hidden" id="breadcrumbs">\n<p class="visuallyhidden" id="breadcrumb__label">You are here:</p>\n<ul aria-labelledby="breadcrumb__label" class="breadcrumbs__list">\n<li class="breadcrumbs__item home">\n<a class="breadcrumbs__link" href="https://www.monash.edu/">Home</a>\n<span aria-hidden="true" class="breadcrumbs__divider">|</span>\n</li>\n<li class="breadcrumbs__item">\n<a class="breadcrumbs__link" href="https://www.monash.edu/study">Study</a>\n<span aria-hidden="true" class="breadcrumbs__divider">|</span>\n</li>\n<li class="breadcrumbs__item">\n<a class="breadcrumbs__link" href="/pubs/2019handbooks/">2019 Handbooks</a>\n<span aria-hidden="true" class="breadcrumbs__divider">|</span>\n</li>\n<li class="breadcrumbs__item breadcrumbs__current"><a class="breadcrumbs__link" href="/pubs/2019handbooks/units">Units</a></li>\n</ul>\n</nav>\n<div class="hbk-banner-box">\n<h1 class="banner_ada"><span class="u

Now, we first need to divide the entire text into chunks of units using a regular expression, which is as done below. 
Each unit tag starts with a pattern `<div class=\"content-inner__main\">` and ends with `<!-- \/.content-inner__main --><\/div>` and hence the below regex has been used with capturing groups to get all the units.

In [3]:
units = re.findall("<div class=\"content-inner__main\">((.|\n)*?)<!-- \/.content-inner__main --><\/div>", my_str)
print("Number of units found are: ",len(units))

Number of units found are:  400


Lets have a look at the first element of the list.

In [4]:
units[0]

('\n<!-- breadcrumbs -->\n<nav class="breadcrumbs mobile-hidden" id="breadcrumbs">\n<p class="visuallyhidden" id="breadcrumb__label">You are here:</p>\n<ul aria-labelledby="breadcrumb__label" class="breadcrumbs__list">\n<li class="breadcrumbs__item home">\n<a class="breadcrumbs__link" href="https://www.monash.edu/">Home</a>\n<span aria-hidden="true" class="breadcrumbs__divider">|</span>\n</li>\n<li class="breadcrumbs__item">\n<a class="breadcrumbs__link" href="https://www.monash.edu/study">Study</a>\n<span aria-hidden="true" class="breadcrumbs__divider">|</span>\n</li>\n<li class="breadcrumbs__item">\n<a class="breadcrumbs__link" href="/pubs/2019handbooks/">2019 Handbooks</a>\n<span aria-hidden="true" class="breadcrumbs__divider">|</span>\n</li>\n<li class="breadcrumbs__item breadcrumbs__current"><a class="breadcrumbs__link" href="/pubs/2019handbooks/units">Units</a></li>\n</ul>\n</nav>\n<div class="hbk-banner-box">\n<h1 class="banner_ada"><span class="unitcode">DIS2907</span> - 3D ani

We have received a list of tuples from the regex for each unit. We convert it to a list of strings by using the below code. 

In [5]:
units_new = [str(x) for x in units]

## 4. Extract useful information from units

Now, the data that would be captured from each unit will be stored in a dictionary. The whole process of the assignment revolves around the dictionary being generated correctly so that it can be dumped into json and xml in the required format. 
We initialise the dictionary in the below way.

In [6]:
unit_guide = {'units': {'unit': []}}

We define the below utility functions to help in extracting the relavant functions. Comments have been provided in the functions below to explain the functionality.

Note: There are some units in the data, that are of length 8, they have been included in the data and the regex is shown below for them. 

In [7]:
# Below is a utility function which converts a list of tuples to a list of strings.
# Parameter: list_tuples - list of tuples
# Returns: list of strings
def convert_to_list(list_tuples):
    return ([str(x) for x in list_tuples])


# Below is a function which extracts the line that contains unit code and 
# title details from the passed string .
# Parameter: test_str - string to search
# Returns: string: matched unit line
def match_unit_line(test_str):
    rexpr = "span class=\"unitcode\">((.|\n)*?)<\/h1>"
    matchobj = re.findall(rexpr, test_str)
    return str(convert_to_list(matchobj))


# Below is a function which extracts the unit code from the passed string.
# Parameter: test_str - string to search
# Returns: string: matched unit code
def match_unit_code(test_str):
    rexpr = "[A-Z]{3}[0-9]{4}|[A-Z]{4}[0-9]{4}"
    matchobj = re.findall(rexpr, test_str)
    return set(matchobj)


# Below is a function which extracts the unit title from the passed string.
# Parameter: test_str - string to search
# Returns: string: matched unit title
def match_unit_title(test_str):
    rexpr = "(?:[A-Z]{3}[0-9]{4}|[A-Z]{4}[0-9]{4})\<\/span> \- (.*)<span class=\"hbk-archive-only\">"
    matchobj = str(re.findall(rexpr, test_str)).strip('[]\'')
    return matchobj


# Below is a function which extracts the synopsis from the passed string.
# Parameter: test_str - string to search
# Returns: string: matched synopsis
def match_synopsis(test_str):
    rexpr = "<h2 class=\"hbk-heading\">Synopsis<\/h2>((.|\n)*?)<\/div>"
    matchobj = str(convert_to_list(re.findall(rexpr, test_str)))    
    rexpr = "<p>(.*?)<\/p>"
    req = re.findall(rexpr, matchobj)
    if(len(req) != 0):
        req = str(req[0]).strip('[]\'')
        req = re.sub('(\<.*?\>)', ' ', req)
    else:
        req = ""
    return req


# Below is a function which extracts the Prerequisites from the passed string.
# Parameter: test_str - string to search
# Returns: string: matched Prerequisites
def match_pre_req(test_str):
    rexpr = "<p class=\"hbk-preamble-heading\">Prerequisites<\/p>((.|\n)*?)<\/p>"
    matchobj = str(convert_to_list(re.findall(rexpr, test_str)))
    rexpr = "<p class=\"hbk-preamble-heading\">Co-requisites<\/p>((.|\n)*?)<\/p>"
    matchobj = matchobj + str(convert_to_list(re.findall(rexpr, test_str)))
    unit_codes = list(match_unit_code(matchobj))
    #print(unit_codes)
    return unit_codes


# Below is a function which extracts the Prohibitions from the passed string.
# Parameter: test_str - string to search
# Returns: string: matched Prohibitions
def match_prohibitions(test_str):
    rexpr = "<p class=\"hbk-preamble-heading\">Prohibitions<\/p>((.|\n)*?)<\/p>"
    matchobj = str(convert_to_list(re.findall(rexpr, test_str)))
    unit_codes = list(match_unit_code(matchobj))
    return unit_codes


# Below is a function which extracts the Requirements from the passed string.
# Parameter: test_str - string to search
# Returns: string: matched Requirements
def match_requirements(test_str):
    rexpr = "<h2 class=\"hbk-heading\">Assessment<\/h2>((.|\n)*?)<\/div>"
    matchobj = str(convert_to_list(re.findall(rexpr, test_str)))
    rexpr = "<p>(.*?)<\/p>"
    req = list(re.findall(rexpr, matchobj))
    return req


# Below is a function which extracts the Outcomes from the passed string.
# Parameter: test_str - string to search
# Returns: string: matched Outcomes
def match_outcomes(test_str):
    rexpr = "<h2 class=\"hbk-heading\">Outcomes<\/h2>((.|\n)*?)<\/div>"
    matchobj = str(convert_to_list(re.findall(rexpr, test_str)))
    rexpr = "<li>(.*?)<\/li>"
    out = re.findall(rexpr, matchobj)
    return out


# Below is a function which extracts the Chief Examiners from the passed string.
# Parameter: test_str - string to search
# Returns: string: matched Chief Examiners
def match_ce(test_str):
    rexpr = "<p class=\"hbk-highlight-heading\">Chief examiner\(s\)<\/p>((.|\n)*?)<\/p>"
    matchobj = str(convert_to_list(re.findall(rexpr, test_str)))
    rexpr = "\">(.*?)<\/a>"
    ce = list(re.findall(rexpr, matchobj))
    return ce


# Below is a function is to remove unwanted characters from the passed string.
# Parameter: test_str - string to change
# Returns: string: updated string
def remove_extra_chars(temp_str):
    temp_str = re.sub(r'[\\\']+', '\'',temp_str)
    temp_str = re.sub('<span.*?html\\\">', '',temp_str)
    temp_str = re.sub('<a class.*?html\\\">', '',temp_str)
    temp_str = re.sub('&gt;', '>',temp_str)
    temp_str = re.sub('(\<.*?\>)', ' ', temp_str)
    temp_str = re.sub('<p>|<\/p>|<ul>|<li>|<\/ul>|<\/li>|<\/a>|<\/span>', '',temp_str)
    return temp_str

Now we need to iterate over the list of unit chunks that we had created and use the above functions to extract the relavant information. Each unit would be stored in a dictionary and then it would be appended to the main dictionary variable which was created above i.e. `unit_guide`

In [8]:
for unit in units_new:
    # This is the dictionary for each unit and intialised as below. 
    unit_dict = {"@id": "", "title": "", "synopsis": "", "pre_requistics": "", 
                "prohibisions": "", "requirements": "", "outcomes": {"outcome": []},
                "chief_examiners": ""}
    
    # Match the unit line. 
    unit_line = match_unit_line(unit)
    
    # Extract the unit code from the matched unit line.
    unit_code = str(match_unit_code(unit_line)).strip('{}\'')
    unit_dict["@id"] = unit_code

    # Extract the unit title from the matched unit line.
    unit_title = match_unit_title(unit_line)
    unit_dict["title"] = unit_title
    
    # Extract the synopsis from the unit chunk.
    synopsis = match_synopsis(unit)
    if(synopsis.strip() != ""):
        synopsis = remove_extra_chars(synopsis)
    unit_dict["synopsis"] = synopsis
    
    # Extract the Prerequisites from the unit chunk.
    pre_req = match_pre_req(unit)
    if(len(pre_req) == 0):
        unit_dict["pre_requistics"] = "NA"
    elif(len(pre_req) == 1):
        unit_dict["pre_requistics"] = {"pre_requistic": pre_req[0]}
    else:
        unit_dict["pre_requistics"] = {"pre_requistic": pre_req}
    
    # Extract the Prohibitions from the unit chunk.
    prohib = match_prohibitions(unit)
    if(len(prohib) == 0):
        unit_dict["prohibisions"] = "NA"
    elif(len(prohib) == 1):
        unit_dict["prohibisions"] = {"prohibision": prohib[0]}
    else:
        unit_dict["prohibisions"] = {"prohibision": prohib}
    
    # Extract the Requirements from the unit chunk.
    req = match_requirements(unit)
    req = list(map(remove_extra_chars, req))
    if(len(req) == 0):
        unit_dict["requirements"] = "NA"
    elif(len(req) == 1):
        unit_dict["requirements"] = {"requirement": req[0]}
    else:
        unit_dict["requirements"] = {"requirement": req}
    
    # Extract the Outcomes from the unit chunk.
    outcomes = match_outcomes(unit)
    outcomes = list(map(remove_extra_chars, outcomes))
    unit_dict["outcomes"]["outcome"] = outcomes
    
    # Extract the Chief Examiners from the unit chunk.
    ce = match_ce(unit)
    ce = list(map(remove_extra_chars, ce))
    if(len(ce) == 0):
        unit_dict["chief_examiners"] = "TBA"
    elif(len(ce) == 1):
        unit_dict["chief_examiners"] = {"chief_examiner": ce[0]}
    else:
        unit_dict["chief_examiners"] = {"chief_examiner": ce}
        
    # In the end, append the unit to the main dictionary and continue with the next chunk.
    unit_guide["units"]["unit"].append(unit_dict)
    

Below we check the number of units that were captured in the dictionary. 

In [9]:
print("Number of units in dictionary: ",len(unit_guide['units']['unit']))

Number of units in dictionary:  400


Lets have a look at the first unit as how it looks like in the clean form in the dictionary. 

In [10]:
print(unit_guide['units']['unit'][0])

{'@id': 'DIS2907', 'title': '3D animation', 'synopsis': 'This unit introduces students to time-based imaging for 3D design and imaging. It develops sequentially upon the skills and knowledge of 3D imaging by providing a focus upon advanced asset creation and animation. Students are introduced to the range of options and controls available through 3D imaging and modeling software applications, towards skillfully creating, manipulating and animating 3D objects, manipulating camera properties and viewer perspectives. Demonstrations and lectures are given on conceiving, designing and producing animation sequences for inclusion into multimedia presentations with consideration for delivery methods and the merits of different media types and formats.', 'pre_requistics': {'pre_requistic': ['DIS1911', 'OHS1000']}, 'prohibisions': 'NA', 'requirements': {'requirement': ['Exercises (30%)', 'Project (60%)', 'Journal (10%)']}, 'outcomes': {'outcome': ['Design and develop defined 3D models with appli

## 5. Dump to json

Once we have the dictionary ready, to create the json file we can use the json package as below with an indent of 5 to be in sync with the test data that was given. 

In [11]:
with open('29620163.json', 'w') as outfile:  
    json.dump(unit_guide, outfile, indent=5)

With this, the json file is generated and stored as needed.

We can use the below code to verify that the json generated is getting parsed properly. 

In [12]:
with open('29620163.json') as f:
    data = json.load(f)
print("Number of units in json file: ",len(data['units']['unit']))

Number of units in json file:  400


## 6. Dump to xml

Below function is created to reuse and dump all the tags as needed in the xml file.

In [13]:
# Function to format based on the tags and return the combined string to be written to file.
# Parameter: unit - dictionary containing the unid data
#            main_tag: the main tag of the xml
#            sub_tag: the nested tag of the main tag
# Returns: output: combined string to written to file
def write_data_xml(unit, main_tag, sub_tag):
    output = ""
    output += "<" + main_tag + ">"
    if(unit[main_tag] == "NA"):
        output += " NA "
    elif(unit[main_tag] == "TBA"):
        output += " TBA "
    else:
        if(isinstance(unit[main_tag][sub_tag], str)):
                output += "\n"
                output += "<" + sub_tag + ">" + unit[main_tag][sub_tag] + "<" + "/" + sub_tag + ">"
        else:
            output += "\n"
            for item in unit[main_tag][sub_tag]:
                output += "<" + sub_tag + ">"  + item + "<" + "/" + sub_tag + ">"
    output += "</" + main_tag + ">\n"
    
    # The below substitution is done, because '&' symbol is a special character for xml.
    output = re.sub('&', '&#38;', output, re.M)
    
    return output

Now, we need to open the output file, and iterate over the abpve created dictionary and keep dumping the items into the file. The below code does that.

In [14]:
with open('29620163.xml', 'w') as outfile:
    outfile.write("<?xml version=\"1.0\" encoding=\"UTF-8\" ?>\n")
    outfile.write("<units>\n")
    for key, val in unit_guide.items():
        units = val['unit']
        for unit in units:
            outfile.write("<unit id='"+unit["@id"] +"'>\n")
            
            outfile.write("<title> "+unit["title"] +"</title>\n")
            
            outfile.write("<synopsis> "+unit["synopsis"] +"</synopsis>\n")
            
            outfile.write(write_data_xml(unit, "pre_requistics", "pre_requistic"))
            
            outfile.write(write_data_xml(unit, "prohibisions", "prohibision"))

            outfile.write(write_data_xml(unit, "requirements", "requirement"))
            
            outfile.write(write_data_xml(unit, "outcomes", "outcome"))
            
            outfile.write(write_data_xml(unit, "chief_examiners", "chief_examiner"))
            
            outfile.write("</unit>\n")
            
    outfile.write("</units>\n")

With this, we have created the needed xml file as well. 

We can verify that the generated xml file can be parsed properly, we can use the below code. Since it uses an external library, the code has been commented so as to not violate any conditions of this assessment. 

In [15]:
# import xml.etree.ElementTree as ET
# root = ET.parse('29620163.xml').getroot()
# print("Number of units in xml file: ",len(root))

## 7. Summary
This assessment measured the data extraction capabilities using Python. Starting from reading the raw text, first the unit chunks are extracted using Regular Expression. Then using various other regex all the relavant information is pulled out. 
To dump the data to json file, json library comes in very handy.
To dump the data to xml file, there are external libraries available which can be used but it has been done by iterating over the data and manually writing it to the file. 
Now, the parsed and structured data can be used by anyone to do some kind of text analytics or for any other relavant application.

## 7. Observations
* There is spelling mistake in the Prerequisites tags. THey are misspelled as 'pre_requistics' and 'pre_requistic', which can be corrected.
* In the sample xml file give, at a few places there are unnecessary new lines. 
* In the Synopsis section, at few places there are multiple paragraphs, but only the first one has been extracted based on the test file provided.

## 8. References
- Website to verify Regular Expressions https://regex101.com/
- Monash Moodle for FIT5196 https://lms.monash.edu/course/view.php?id=50799
- Python JSON https://www.w3schools.com/python/python_json.asp